## Inicializando o Dataset

Autor: *Thiago Vilarinho Lemes*

Data: *13/08/2022*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import date
import numpy as np

In [ ]:
WORK_DIR = 'path_dataset'
DATA_DIR = os.path.join(WORK_DIR, 'ifood')

In [ ]:
df_orders = pd.read_csv(os.path.join(DATA_DIR, 'orders.csv'))

## Criando as Features e Target 

In [ ]:
# Criando o Dataframe df_features_train 2019-06-01 (inclusivo) até 2019-07-01 (exclusivo)
df_features_train = df_orders.query("order_timestamp_local >= '2019-06-01' & order_timestamp_local < '2019-07-01'")

In [ ]:
# Verificando o tamanho do Dataframe df_features_train
df_features_train.shape

(88451, 43)

In [ ]:
# Criando o Dataframe df_target_train 2019-07-01 (inclusivo) até 2019-08-01 (exclusivo)
df_target_train = (
  df_orders
    # Selecionando o periodo de  2019-07-01 (inclusivo) até 2019-08-01 (exclusivo)
    .query("order_timestamp_local >= '2019-07-01' & order_timestamp_local < '2019-08-01'")
    # Selecionando somente a features customer_id, que será a target
    .filter(['customer_id'])
    # Apagando as features duplicadas
    .drop_duplicates()
)

In [ ]:
# Verificando o tamanho do Dataframe df_abt_train
df_target_train.shape

(18441, 1)

In [ ]:
# Verificando a lista de features
df_target_train.columns.to_list()

['customer_id']

## Criando ABT Train

In [ ]:
# Criando o Dataframe df_abt_train
df_abt_train = (
    df_features_train
    # Agrupando por customer_id, unindo as features com o mesmo valor
    .groupby('customer_id')
    # Criando as features fazendo a somatória das mesmas, somando os valores
    .agg(
      # Criando a feature receita_1m
      receita_1m=('paid_amount', 'sum'),
      # Criando a feature qtd_pedidos_1m
      qtd_pedidos_1m=('order_id', 'nunique'),
      # Criando a feature data_ultima_venda
      data_ultima_venda=('order_timestamp_local', 'max')
    )
    # Resetando o index
    .reset_index()
)

In [ ]:
# Verificando o tamanho do Dataframe df_abt_train
df_abt_train.shape

(30079, 4)

In [ ]:
# Verificando a lista de colunas
df_abt_train.columns.to_list()

['customer_id', 'receita_1m', 'qtd_pedidos_1m', 'data_ultima_venda']

In [ ]:
# Alerando o Dataframe df_abt_train
df_abt_train = (
    df_abt_train
      # Alterando a feature data_ultima_venda para o tipo datetime
      .assign(data_ultima_venda = pd.to_datetime(df_abt_train['data_ultima_venda'], utc=True))
      # Alterando a feature data_ref para o tipo datetime
      .assign(data_ref = pd.to_datetime('2019-07-01', utc=True))
      # Criando a feature recencia da subtração entre as features df['data_ref'] - df['data_ultima_venda'], df é só um nome simbólico
      .assign(recencia = lambda df: (df['data_ref'] - df['data_ultima_venda']).dt.days)
      # Filtrando as features para o Dataframe
      .filter([
          'data_ref', 
          'customer_id', 
          'receita_1m', 
          'qtd_pedidos_1m', 
          'recencia'])
)

In [ ]:
# Verificando o tamanho do Dataframe df_abt_train
df_abt_train.shape

(30079, 5)

In [ ]:
# Verificando a lista de colunas
df_abt_train.columns.to_list()

['data_ref', 'customer_id', 'receita_1m', 'qtd_pedidos_1m', 'recencia']

In [ ]:
# Fazendo o merge entre os Dataframes df_abt_train e o df_target_train
df_abt_train = (
    df_abt_train
    # Unindo os Dataframes df_abt_train e df_target_train que contenham a feature customer_id em comum
    # criando a feature _merge, que por sua vez irá conter left_only e only
    .merge(df_target_train, on='customer_id', how='left', indicator=True)
)

In [ ]:
# Verificando o tamanho do Dataframe df_abt_train
df_abt_train.shape

(30079, 6)

In [ ]:
# Verificando a lista de colunas
df_abt_train.columns.to_list()

['data_ref',
 'customer_id',
 'receita_1m',
 'qtd_pedidos_1m',
 'recencia',
 '_merge']

In [ ]:
df_abt_train = (
    df_abt_train
      # Criando a feature churn_next_month a  partir da feature _merge, 
      # clientes que irão dar churn no próximo mês, churm - que não vão comprar no próximo mês 
      # A feature churn_next_month irá substituir a feature _merge por valores
      # left_only = 1 --> churn
      # onlu = 0      --> não churn
      .assign(churn_next_month = lambda df: np.where(df['_merge']=='left_only', 1, 0))
      # Apaga a coluna _merge
      .drop('_merge', axis=1)
)

In [ ]:
# Verificando o tamanho do Dataframe df_abt_train
df_abt_train.shape

(30079, 6)

In [ ]:
# Verificando a lista de colunas
df_abt_train.columns.to_list()

['data_ref',
 'customer_id',
 'receita_1m',
 'qtd_pedidos_1m',
 'recencia',
 'churn_next_month']

In [ ]:
# Verificando a distribuição da feature churn_next_month
df_abt_train['churn_next_month'].value_counts(normalize=True)

0    0.613086
1    0.386914
Name: churn_next_month, dtype: float64

In [ ]:
# Verificando o Dataframe df_abt_train
df_abt_train.shape

(30079, 6)

## Criando a ABT Out of Time avançando 1 mês

2019-07-01 (inclusivo) até 2019-08-01 (exclusivo)

## Criando a Feature e a Target Out of Time

In [ ]:
# Criando o Dataframe df_features_train 2019-07-01 (inclusivo) até 2019-08-01 (exclusivo)
df_features_oot = df_orders.query("order_timestamp_local >= '2019-07-01' & order_timestamp_local < '2019-08-01'")

In [ ]:
# Verificando o tamanho do Dataframe df_features_train
df_features_oot.shape

(69945, 43)

In [ ]:
# Criando o Dataframe df_target_train 2019-07-01 (inclusivo) até 2019-08-01 (exclusivo)
df_target_oot = (
  df_orders
    # Selecionando o periodo de  2019-07-01 (inclusivo) até 2019-08-01 (exclusivo)
    .query("order_timestamp_local >= '2019-08-01' & order_timestamp_local < '2019-09-01'")
    # Selecionando somente a features customer_id, que será a target
    .filter(['customer_id'])
    # Apagando as features duplicadas
    .drop_duplicates()
)

In [ ]:
# Verificando o tamanho do Dataframe df_features_train
df_target_oot.shape

(17319, 1)

## Criando a ABT Out of Time

In [ ]:
# Criando o Dataframe df_abt_train
df_abt_oot = (
    df_features_oot
    # Agrupando por customer_id, unindo as features com o mesmo valor
    .groupby('customer_id')
    # Criando as features fazendo a somatória das mesmas, somando os valores
    .agg(
      # Criando a feature receita_1m
      receita_1m=('paid_amount', 'sum'),
      # Criando a feature qtd_pedidos_1m
      qtd_pedidos_1m=('order_id', 'nunique'),
      # Criando a feature data_ultima_venda
      data_ultima_venda=('order_timestamp_local', 'max')
    )
    # Resetando o index
    .reset_index()
)

In [ ]:
# Verificando o tamanho do Dataframe df_features_train
df_abt_oot.shape

(18441, 4)

In [ ]:
# Alerando o Dataframe df_abt_train
df_abt_oot = (
    df_abt_oot
      # Alterando a feature data_ultima_venda para o tipo datetime
      .assign(data_ultima_venda = pd.to_datetime(df_abt_oot['data_ultima_venda'], utc=True))
      # Alterando a feature data_ref para o tipo datetime
      .assign(data_ref = pd.to_datetime('2019-08-01', utc=True))
      # Criando a feature recencia da subtração entre as features df['data_ref'] - df['data_ultima_venda'], df é só um nome simbólico
      .assign(recencia = lambda df: (df['data_ref'] - df['data_ultima_venda']).dt.days)
      # Filtrando as features para o Dataframe
      .filter([
          'data_ref', 
          'customer_id', 
          'receita_1m', 
          'qtd_pedidos_1m', 
          'recencia'])
)

In [ ]:
# Verificando o tamanho do Dataframe df_features_train
df_abt_oot.shape

(18441, 5)

In [ ]:
# Verificando a lista de colunas
df_abt_oot.columns.to_list()

['data_ref', 'customer_id', 'receita_1m', 'qtd_pedidos_1m', 'recencia']

In [ ]:
# Fazendo o merge entre os Dataframes df_abt_train e o df_target_train
df_abt_oot = (
    df_abt_oot
    # Unindo os Dataframes df_abt_train e df_target_train que contenham a feature customer_id em comum
    # criando a feature _merge, que por sua vez irá conte left_only e only
    .merge(df_target_oot, on='customer_id', how='left', indicator=True)
)

In [ ]:
# Verificando o tamanho do Dataframe df_features_train
df_abt_oot.shape

(18441, 6)

In [ ]:
# Verificando a lista de colunas
df_abt_oot.columns.to_list()

['data_ref',
 'customer_id',
 'receita_1m',
 'qtd_pedidos_1m',
 'recencia',
 '_merge']

In [ ]:
df_abt_oot = (
    df_abt_oot
      # Criando a feature churn_next_month a  partir da feature _merge, 
      # clientes que irão dar churn no próximo mês, churm - que não vão comprar no próximo mês 
      # A feature churn_next_month irá substituir a feature _merge por valores
      # left_only = 1 --> churn
      # onlu = 0      --> não churn
      .assign(churn_next_month = lambda df: np.where(df['_merge']=='left_only', 1, 0))
      # Apaga a coluna _merge
      .drop('_merge', axis=1)
)

In [ ]:
# Verificando o tamanho do Dataframe df_features_train
df_abt_oot.shape

(18441, 6)

In [ ]:
# Verificando a lista de colunas
df_abt_oot.columns.to_list()

['data_ref',
 'customer_id',
 'receita_1m',
 'qtd_pedidos_1m',
 'recencia',
 'churn_next_month']

In [ ]:
# Verificando a distribuição da feature churn_next_month
df_abt_oot['churn_next_month'].value_counts(normalize=True)

0    0.759558
1    0.240442
Name: churn_next_month, dtype: float64

In [ ]:
df_abt_oot.shape

(18441, 6)

## Estudo dos valores faltantes entre os Dataframes df_abt_train e df_abt_oot

In [ ]:
# Verificando os valore nulos do Dataframe df_abt_train
df_abt_train.isnull().sum()

data_ref            0
customer_id         0
receita_1m          0
qtd_pedidos_1m      0
recencia            0
churn_next_month    0
dtype: int64

In [ ]:
# Verificando os valore nulos do Dataframe df_abt_oot
df_abt_oot.isnull().sum()

data_ref            0
customer_id         0
receita_1m          0
qtd_pedidos_1m      0
recencia            0
churn_next_month    0
dtype: int64